# Bright Pixel

Skriv en tekst, som omsættes til et billede

## Opgave 1

* Sørg for at de korrekte nøgler er skrevet ind i .env filen.
* Indlæs miljøvariable (environment variables).


In [ ]:
# Import namespaces
from dotenv import load_dotenv
import openai
import os
from uuid import uuid4
import urllib.request

# Set default save path
save_path = "./images/" # Ændre denne til den mappe du vil gemme billederne i

# Get Configuration Settings
load_dotenv()
openai_key = os.getenv('<YOUR_OPENAI_KEY>')

# Configure openai
openai.api_key = openai_key

## Opgave 2

* Gennemlæs koden nedenunder, som kalder DALL-E API'et
* Tjek response-objektet. Undersøg de andre parametre i response-objektet
* Se hvor billedfilerne gemmes
    * Tjek navngivningen for billedet - hvorfor anvendes uuid?
* Test koden
* Justér på parametrene

In [ ]:
# Create image function
def create_image(request, save=True):
    response = openai.Image.create(
        prompt=request,
        n=1,
        size="1024x1024"
    )
    image_url = response['data'][0]['url']
    full_save_path = save_path + f"openai_{uuid4()}.png"
    urllib.request.urlretrieve(image_url, full_save_path)
    print(f"\nImage created successfully! Saved to: {full_save_path}\n")
    return full_save_path

In [ ]:
# Test koden
command = "A dog with a red collar is running through a field."
create_image(command)

## Opgave 3

* Gennemlæse koden nedenunder, hvor al ovenstående kode samles

In [ ]:
# Main function
def main():
    while True:
        print("Waiting for user input (write 'quit' to exit)...")
        request = input()
    
        if request == "quit":
            print("Exiting...")
            break

        # Create image
        create_image(request)  

# Run main
if __name__ == "__main__":
    main()

## Opgave 4

* Saml al kode i én fil
* Skriv filen til main.py
* Brug en terminal til at køre filen

In [ ]:
# %%writefile main.py
# Sammensæt selv koden

# Ekstra opgave

* Brug Azure Computer Vision til at analysere billederne

# Løsningsforslag

In [ ]:
# %%writefile main.py

# Import namespaces
from dotenv import load_dotenv
import openai
import os
from uuid import uuid4
import urllib.request

# Set default save path
save_path = "./images/"

# Get Configuration Settings
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Configure openai
openai.api_key = openai_key

# Create image function
def create_image(request, save=True):
    response = openai.Image.create(
        prompt=request,
        n=1,
        size="1024x1024"
    )
    image_url = response['data'][0]['url']
    full_save_path = save_path + f"openai_{uuid4()}.png"
    urllib.request.urlretrieve(image_url, full_save_path)
    print(f"\nImage created successfully! Saved to: {full_save_path}\n")
    return full_save_path

# Main function
def main():
    while True:
        print("Waiting for user input (write 'quit' to exit)...")
        request = input()
    
        if request == "quit":
            print("Exiting...")
            break

        # Create image
        create_image(request)  

# Run main
if __name__ == "__main__":
    main()